In [2]:
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

## Hyperparameters

In [3]:
# 全結合層のためにweightsとbiasesをランダムに初期化するためのhelper function
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# サイズを"sizes"とした全結合層の全ての層を初期化
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 8
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.PRNGKey(0))

# Auto-batching predictions

1枚の画像の例のために予測関数を定義しよう。。パフォーマンスの低下なしにミニバッチを自動的に扱うためにJAXの`vmap`関数を使う。

In [4]:
from jax.scipy.special import logsumexp

def relu(x):
  return jnp.maximum(0, x)

def predict(params, image):
  # exampleごとの予測
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = relu(outputs)

  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits - logsumexp(logits)

今作った予測関数が1枚の画像でのみ動くことを確かめよう

In [5]:
# 1枚の画像で動く
random_flattenend_image = random.normal(random.PRNGKey(1), (28 * 28,))
preds = predict(params, random_flattenend_image)
print(preds.shape)

(10,)


In [6]:
# バッチでは動かない
random_flattenend_images = random.normal(random.PRNGKey(1), (10, 28 * 28))
try:
  preds = predict(params, random_flattenend_images)
except TypeError:
  print('Invalid shapes!')

Invalid shapes!


In [7]:
# バッチを扱えるように `vmap` を使ってアップグレードしよう

# 予測関数のバッチ版を作る
batched_predict = vmap(predict, in_axes=(None, 0))

# `batched_predict` は `predict` と同じcallサインを持つ
batched_preds = batched_predict(params, random_flattenend_images)
print(batched_preds.shape)

(10, 10)


ここで、私たちのニューラルネットワークを定義し、それを学習するのに必要な全ての材料が揃った。自動バッチ版の予測関数`predict`を構築した。これは損失関数の中で使えるはずのものである。ニューラルネットワークのパラメータごとのlossの微分を行うために`grad`を使う。最後に、全てをスピードアップするために`jit`を使用する。

## Utility and loss functions

In [8]:
def one_hot(x, k, dtype=jnp.float32):
  """ x についての、サイズ k のone-hotエンコーディングを作成"""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)

def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = batched_predict(params, images)
  return -jnp.mean(preds * targets)

@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]

## Data Loading with PyTorch
JAXはtransformationsに集中し、かつNumPyをバックとしているため、私たちはデータローディングやmunging（データをいろいろ加工すること）はJAXライブラリには含めていない。優れたデータローダーがすでにたくさんあるため、そういったものを再発明する代わりにそれらを使おう。PyTorchのデータローダーを使い、NumPy配列で動く簡単な機能を作ってみる。

In [9]:
import numpy as np
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class NumpyLoader(data.DataLoader):
  def __init__(self, dataset, batch_size=1,
               shuffle=False, sampler=None,
               batch_sampler=None, num_workers=0,
               pin_memory=False, drop_last=False,
               timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
          batch_size=batch_size,
          shuffle=shuffle,
          sampler=sampler,
          batch_sampler=batch_sampler,
          num_workers=num_workers,
          collate_fn=numpy_collate,
          pin_memory=pin_memory,
          drop_last=drop_last,
          timeout=timeout,
          worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))

/Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F064E54E-B9B1-3EC9-9B6C-7AF012333C35> /Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <0BB917BA-0D8F-3429-B798-476DA4A619BD> /Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [10]:
# torch datasetを使ってdatasetを定義する
mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0)

In [12]:
# フルのtrain datasetをゲット（学習中の精度を確認するため）
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# フルのtest datasetをゲット
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

/Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/yukik/Work/ML/ML-Library/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## Training Loop

In [13]:
import time

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    params = update(params, x, y)
  epoch_time = time.time() - start_time
  
  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print(f"Epoch {epoch} in {epoch_time:0.2f} sec")
  print(f"Training set accuracy {train_acc}")
  print(f"Test set accuracy {test_acc}")

Epoch 0 in 2.24 sec
Training set accuracy 0.9157500267028809
Test set accuracy 0.9199000000953674
Epoch 1 in 2.03 sec
Training set accuracy 0.937166690826416
Test set accuracy 0.9384999871253967
Epoch 2 in 2.00 sec
Training set accuracy 0.9492499828338623
Test set accuracy 0.946899950504303
Epoch 3 in 2.13 sec
Training set accuracy 0.9568166732788086
Test set accuracy 0.9532999992370605
Epoch 4 in 1.96 sec
Training set accuracy 0.963100016117096
Test set accuracy 0.9573999643325806
Epoch 5 in 1.96 sec
Training set accuracy 0.9673833250999451
Test set accuracy 0.9614999890327454
Epoch 6 in 1.99 sec
Training set accuracy 0.9708166718482971
Test set accuracy 0.9648999571800232
Epoch 7 in 1.96 sec
Training set accuracy 0.9736166596412659
Test set accuracy 0.9667999744415283
